In [1]:
# code by Peter Solis

In [2]:
# dependencies
import pandas as pd
from datetime import datetime as dt

In [3]:
# MERGING - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# pull 3 CSVs to dataframes
IDs_df = pd.read_csv('../Raw Data/2000s_US_IDs.csv')
credits_df = pd.read_csv('../Raw Data/movie_credits.csv')
details_df = pd.read_csv('../Raw Data/movie_details.csv')

details_df.head()

,Unnamed: 0,Movie ID,Budget,Revenue,Genre 1,Genre 2,Genre 3,Genre 4,Genre 5,Release Date,Run Time
0,0,11688,100000000,169327687,Adventure,Animation,Comedy,Family,Fantasy,2000-12-15,78
1,1,4247,19000000,278019771,Comedy,NaN,NaN,NaN,NaN,2000-07-07,88
2,2,9600,30000000,173959438,Crime,Comedy,NaN,NaN,NaN,2000-05-31,98
3,3,98,103000000,465361176,Action,Drama,Adventure,NaN,NaN,2000-05-04,155
4,4,3134,0,940944,Crime,Drama,Romance,Thriller,NaN,2000-06-28,77


In [4]:
# delete extra index column in credits & details
credits_df = credits_df.drop(columns=['Unnamed: 0'])
details_df = details_df.drop(columns=['Unnamed: 0'])

details_df.head()

,Movie ID,Budget,Revenue,Genre 1,Genre 2,Genre 3,Genre 4,Genre 5,Release Date,Run Time
0,11688,100000000,169327687,Adventure,Animation,Comedy,Family,Fantasy,2000-12-15,78
1,4247,19000000,278019771,Comedy,NaN,NaN,NaN,NaN,2000-07-07,88
2,9600,30000000,173959438,Crime,Comedy,NaN,NaN,NaN,2000-05-31,98
3,98,103000000,465361176,Action,Drama,Adventure,NaN,NaN,2000-05-04,155
4,3134,0,940944,Crime,Drama,Romance,Thriller,NaN,2000-06-28,77


In [5]:
print(len(IDs_df))
print(len(credits_df))
print(len(details_df))

3168
3168
3168


In [6]:
# merge 3 dataframes
merge_df_1 = pd.merge(IDs_df,
                    details_df,
                    on='Movie ID',
                    how='left')
merge_df_2 = pd.merge(merge_df_1,
                    credits_df,
                    on='Movie ID',
                    how='left')

merge_df_2.shape

(3168, 19)

In [7]:
# CLEANING - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# purge movies that have no budget or revenue data
money_condition = (
    (merge_df_2['Budget'] > 0) & 
    (merge_df_2['Revenue'] > 0))
with_money_df = merge_df_2[money_condition]

with_money_df.shape

(1978, 19)

In [8]:
# parse date
dates = pd.to_datetime(with_money_df['Release Date'],
                       format = '%Y-%m-%d')

with_date_df = with_money_df.drop(columns = ['Release Year', 'Release Date'])
with_date_df['Release Date'] = dates
with_date_df['Release Day'] = dates.dt.day
with_date_df['Release Month'] = dates.dt.month
with_date_df['Release Year'] = dates.dt.year

with_date_df.head()

,Movie ID,Title,Popularity Score,Average Rating,Number of Ratings,Budget,Revenue,Genre 1,Genre 2,Genre 3,...,Genre 5,Run Time,Actor 1,Actor 2,Actor 3,Director,Release Date,Release Day,Release Month,Release Year
0,11688,The Emperor's New Groove,87.669,7.5,6005,100000000,169327687,Adventure,Animation,Comedy,...,Fantasy,78,David Spade,John Goodman,Eartha Kitt,Mark Dindal,2000-12-15,15,12,2000
1,4247,Scary Movie,85.388,6.3,6244,19000000,278019771,Comedy,NaN,NaN,...,NaN,88,Anna Faris,Jon Abrahams,Marlon Wayans,Keenen Ivory Wayans,2000-07-07,7,7,2000
2,9600,Big Momma's House,84.006,5.8,2096,30000000,173959438,Crime,Comedy,NaN,...,NaN,98,Martin Lawrence,Nia Long,Paul Giamatti,Raja Gosnell,2000-05-31,31,5,2000
3,98,Gladiator,72.746,8.2,16703,103000000,465361176,Action,Drama,Adventure,...,NaN,155,Russell Crowe,Joaquin Phoenix,Connie Nielsen,Ridley Scott,2000-05-04,4,5,2000
6,955,Mission: Impossible II,64.621,6.1,5983,125000000,546388105,Adventure,Action,Thriller,...,NaN,123,Tom Cruise,Dougray Scott,Thandiwe Newton,John Woo,2000-05-24,24,5,2000


In [9]:
# no made-for-TV movies
tv_condition = (
    (with_date_df['Genre 1'] != 'TV Movie') |
    (with_date_df['Genre 2'] != 'TV Movie') |
    (with_date_df['Genre 3'] != 'TV Movie') |
    (with_date_df['Genre 4'] != 'TV Movie') |
    (with_date_df['Genre 5'] != 'TV Movie'))
no_tv_df = with_date_df[tv_condition]

no_tv_df.shape

(1978, 21)

In [10]:
# make sure we have actors / some genre
details_condition = (
    (no_tv_df['Actor 1'].notna()) &
    (no_tv_df['Genre 1'].notna()))
has_details_df = no_tv_df[details_condition]

has_details_df.shape

(1978, 21)

In [11]:
# list data types
has_details_df.dtypes

Movie ID                      int64
Title                        object
Popularity Score            float64
Average Rating              float64
Number of Ratings             int64
Budget                        int64
Revenue                       int64
Genre 1                      object
Genre 2                      object
Genre 3                      object
Genre 4                      object
Genre 5                      object
Run Time                      int64
Actor 1                      object
Actor 2                      object
Actor 3                      object
Director                     object
Release Date         datetime64[ns]
Release Day                   int64
Release Month                 int64
Release Year                  int64
dtype: object

In [12]:
# make columns for:
# net revenue 
net_rev = has_details_df['Revenue'] - has_details_df['Budget']
# net revenue as a % over budget (0 = movie broke even with budget)
net_rev_percent = net_rev / has_details_df['Budget']

has_details_df['Net Revenue'] = net_rev
has_details_df['Net Revenue (% Over Budget)'] = net_rev_percent

has_details_df.head()

,Movie ID,Title,Popularity Score,Average Rating,Number of Ratings,Budget,Revenue,Genre 1,Genre 2,Genre 3,...,Actor 1,Actor 2,Actor 3,Director,Release Date,Release Day,Release Month,Release Year,Net Revenue,Net Revenue (% Over Budget)
0,11688,The Emperor's New Groove,87.669,7.5,6005,100000000,169327687,Adventure,Animation,Comedy,...,David Spade,John Goodman,Eartha Kitt,Mark Dindal,2000-12-15,15,12,2000,69327687,0.693277
1,4247,Scary Movie,85.388,6.3,6244,19000000,278019771,Comedy,NaN,NaN,...,Anna Faris,Jon Abrahams,Marlon Wayans,Keenen Ivory Wayans,2000-07-07,7,7,2000,259019771,13.632620
2,9600,Big Momma's House,84.006,5.8,2096,30000000,173959438,Crime,Comedy,NaN,...,Martin Lawrence,Nia Long,Paul Giamatti,Raja Gosnell,2000-05-31,31,5,2000,143959438,4.798648
3,98,Gladiator,72.746,8.2,16703,103000000,465361176,Action,Drama,Adventure,...,Russell Crowe,Joaquin Phoenix,Connie Nielsen,Ridley Scott,2000-05-04,4,5,2000,362361176,3.518070
6,955,Mission: Impossible II,64.621,6.1,5983,125000000,546388105,Adventure,Action,Thriller,...,Tom Cruise,Dougray Scott,Thandiwe Newton,John Woo,2000-05-24,24,5,2000,421388105,3.371105


In [13]:
# runtime of at least 1 hour
runtime_condition = (has_details_df['Run Time'] >= 60)
good_length_df = has_details_df[runtime_condition]

good_length_df.shape

(1978, 23)

In [14]:
# at least 250 user reviews
review_condition = (good_length_df['Number of Ratings'] >= 250)
review_count_df = good_length_df[review_condition]

review_count_df.shape

(1941, 23)

In [15]:
# list columns for reference
review_count_df.columns

Index(['Movie ID', 'Title', 'Popularity Score', 'Average Rating',
       'Number of Ratings', 'Budget', 'Revenue', 'Genre 1', 'Genre 2',
       'Genre 3', 'Genre 4', 'Genre 5', 'Run Time', 'Actor 1', 'Actor 2',
       'Actor 3', 'Director', 'Release Date', 'Release Day', 'Release Month',
       'Release Year', 'Net Revenue', 'Net Revenue (% Over Budget)'],
      dtype='object')

In [16]:
# make final df (sort columns)
final_df = review_count_df[[
    # movie identifiers
    'Movie ID',
    'Title',
    # success-related statistics
    'Average Rating',
    'Number of Ratings',
    'Popularity Score',
    'Revenue',
    'Net Revenue',
    'Net Revenue (% Over Budget)',
    # variables
    'Budget',
    'Release Date',
    'Release Day',
    'Release Month',
    'Release Year',
    'Genre 1',
    'Genre 2',
    'Genre 3',
    'Genre 4',
    'Genre 5',
    'Run Time',
    'Actor 1',
    'Actor 2',
    'Actor 3',
    'Director']]

final_df.shape

(1941, 23)

In [17]:
# rename columns where needed
final_df = final_df.rename(columns = {'Average Rating': 'Average Rating (Out of 10)',
                                      'Popularity Score': 'TMDb Popularity Score',
                                      'Number of Ratings': 'Number of User Ratings',
                                      'Revenue': 'Gross Revenue',
                                      'Run Time': 'Runtime (Minutes)',
                                      'Actor 1': '1st Billing',
                                      'Actor 2': '2nd Billing',
                                      'Actor 3': '3rd Billing'})

In [18]:
# set movie ID to index & export
final_df = final_df.set_index('Movie ID')

final_df.to_csv('../Cleaned Data/cleaned_movie_data.csv')

final_df.head()

,Title,Average Rating (Out of 10),Number of User Ratings,TMDb Popularity Score,Gross Revenue,Net Revenue,Net Revenue (% Over Budget),Budget,Release Date,Release Day,...,Genre 1,Genre 2,Genre 3,Genre 4,Genre 5,Runtime (Minutes),1st Billing,2nd Billing,3rd Billing,Director
Movie ID,,,,,,,,,,,,,,,,,,,,,
11688,The Emperor's New Groove,7.5,6005,87.669,169327687,69327687,0.693277,100000000,2000-12-15,15,...,Adventure,Animation,Comedy,Family,Fantasy,78,David Spade,John Goodman,Eartha Kitt,Mark Dindal
4247,Scary Movie,6.3,6244,85.388,278019771,259019771,13.632620,19000000,2000-07-07,7,...,Comedy,NaN,NaN,NaN,NaN,88,Anna Faris,Jon Abrahams,Marlon Wayans,Keenen Ivory Wayans
9600,Big Momma's House,5.8,2096,84.006,173959438,143959438,4.798648,30000000,2000-05-31,31,...,Crime,Comedy,NaN,NaN,NaN,98,Martin Lawrence,Nia Long,Paul Giamatti,Raja Gosnell
98,Gladiator,8.2,16703,72.746,465361176,362361176,3.518070,103000000,2000-05-04,4,...,Action,Drama,Adventure,NaN,NaN,155,Russell Crowe,Joaquin Phoenix,Connie Nielsen,Ridley Scott
955,Mission: Impossible II,6.1,5983,64.621,546388105,421388105,3.371105,125000000,2000-05-24,24,...,Adventure,Action,Thriller,NaN,NaN,123,Tom Cruise,Dougray Scott,Thandiwe Newton,John Woo
